In [1]:
import requests
from string import ascii_letters
import json
from pprint import pprint
from datetime import datetime
from collections import defaultdict
import logging

In [2]:
#https://www.toptal.com/python/in-depth-python-logging
# different levels of loggins 

In [3]:
logger = logging.getLogger("My Requests App")
logger.setLevel(logging.INFO)

# create a file handler
handler = logging.FileHandler('hello.log')
handler.setLevel(logging.INFO)

# create a logging format
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

# add the handlers to the logger
logger.addHandler(handler)


In [19]:
def get_last_id (url):
    logger = logging.getLogger("My Requests App.get_last_id")
    
    resp = requests.get(url, auth=('admin', 'adminpassword'))
    logger.info('Send GET request to {}'.format(url))
    resp_list = json.loads(resp.text)

    last_id = sorted([i['id'] for i in resp_list])[-1]
    logger.info('Got last ID - {}'.format(last_id))
    #pprint(json.loads(resp.text)[-1])
    return last_id

In [20]:
url = 'http://127.0.0.1:8000/todos/'
last_id = get_last_id(url)

In [33]:
def delete_last_id(url_to_delete):
    logger = logging.getLogger("My Requests App.delete_last_id")
    try:
        delete_response = requests.delete(url_to_delete, timeout=10)
    except requests.exceptions.ConnectionError as e:
        print(str(e), 'here')
        logger.critical('API not on or wrong port')
        return False
    except requests.exceptions.Timeout as e1:
        print(str(e1))
        logger.error('Request took too long to complete')
        return False

    
    if delete_response.status_code == 204:
        print('Successful!', delete_response.status_code, delete_response.reason)
        
    if delete_response.status_code == 404:
        print(delete_response.status_code, delete_response.reason)
        logger.error('URL not found -- {}'.format(url_to_delete))
    
    #return delete_response

In [34]:

url_to_delete = "{}{}/".format(url, last_id)
#print(url_to_delete)
deleted_url = delete_last_id(url_to_delete)

HTTPConnectionPool(host='127.0.0.1', port=8000): Max retries exceeded with url: /todos/57/ (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x10ad00b70>: Failed to establish a new connection: [Errno 61] Connection refused',))


In [124]:
url = 'http://127.0.0.1:8082/todos/'
try:
    r = requests.get(url)
    print(r.status_code, r.reason, r.text)
except requests.exceptions.ConnectionError as e:
    print(str(e))

HTTPConnectionPool(host='127.0.0.1', port=8082): Max retries exceeded with url: /todos/ (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x104999d68>: Failed to establish a new connection: [Errno 61] Connection refused',))


In [111]:
requests.exceptions.Timeout

(404, 'Not Found')

In [110]:
url_to_delete

'http://127.0.0.1:8000/todos/58/'

In [14]:
def create_buncha_post(url):

    count = get_last_id(url) +1
    for i in range(len(ascii_letters)):
        p = requests.post(url, data={'title': 'title_{}'.format(count),  'content': 'content_{}'.format(ascii_letters[i])})
        print(p.status_code, p.reason)
        count+=1

In [84]:
url = 'http://127.0.0.1:8000/todos/'



b'{"id":11,"title":"title9","content":"description_8","created_at":"2018-09-10T00:48:36.011199Z","updated_at":"2018-09-10T00:48:36.011237Z"}'

status code & reason

(204, 'No Content') 
 ^^ when you successfully delete 

(404, 'Not Found')  -- no Exception -- e.g. search for an endpoint that doesnt exist 
^^ try to delete something that doesnt exist 

(404, 'Not Found')  -- with exception 
HTTPConnectionPool(host='127.0.0.1', port=8000): Max retries exceeded with url: /todos/9/ (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x111c38588>: Failed to establish a new connection: [Errno 61] Connection refused',))




______ catch error when api is down 
try:
    delete_response = requests.delete(url_to_delete)
except requests.exceptions.ConnectionError as e:
    print(str(e), 'here')
delete_response.status_code, delete_response.reason



### add one about SSL certs in logging debug 

-- deleting a hostname that doesnt exist  (404 not found) -- no exception  -- make a get request before this for that id & stop if not exist (warn) -- or if GET != 200
-- api is off (404 not found) -- with exception (critical)
-- successful delete (204 - no content) (info)
-- if get not 200 
-- proper authorization -- (403 forbidden)
-- connecting to wrong django port -- but if it was on wrong port would django even be able to respond? 
- request took too long to complete -- exception -- use timeout kwarg on request.delete

